### Import

In [3]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ----- ---------------------------------- 16.8/124.9 MB 86.2 MB/s eta 0:00:02
   ------------- ------------------------- 41.9/124.9 MB 102.8 MB/s eta 0:00:01
   --------------------- ----------------- 67.4/124.9 MB 108.1 MB/s eta 0:00:01
   ---------------------------- ---------- 92.3/124.9 MB 110.4 MB/s eta 0:00:01
   ----------------------------------- -- 117.2/124.9 MB 111.6 MB/s eta 0:00:01
   -------------------------------------- 124.9/124.9 MB 101.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder

import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

### Data Load

In [7]:
train = pd.read_csv('./Data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./Data/test.csv').drop(columns=['ID'])

In [8]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [9]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [10]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [11]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [12]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [13]:
numeric_columns = [col for col in numeric_columns if col in X.columns]

X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_encoded, y, test_size=0.2, random_state=42, stratify=y
)

### Train

XGBoost

In [15]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "lambda": trial.suggest_float("lambda", 1, 10),
        "alpha": trial.suggest_float("alpha", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
    }

    model = xgb.XGBClassifier(
        **params,
        objective="binary:logistic",
        eval_metric="auc",
        use_label_encoder=False,
        verbosity=0,  # 경고 메시지 숨김
        random_state=42,
        n_jobs=-1,
        scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1])
    )

    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
    y_pred = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, y_pred)

    return auc


In [12]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=150, show_progress_bar=True)

[I 2025-02-14 03:01:17,065] A new study created in memory with name: no-name-b8ad6b82-8b8a-4dfb-a2e8-d1b7cded3fcc
  0%|          | 0/150 [00:00<?, ?it/s]C:\Users\Owner\AppData\Local\Temp\ipykernel_848\2214655158.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
Best trial: 0. Best value: 0.737246:   1%|          | 1/150 [00:03<09:07,  3.68s/it]C:\Users\Owner\AppData\Local\Temp\ipykernel_848\2214655158.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),


[I 2025-02-14 03:01:20,742] Trial 0 finished with value: 0.7372457337175858 and parameters: {'n_estimators': 200, 'max_depth': 8, 'learning_rate': 0.04934458314956417, 'subsample': 0.5122009348851322, 'colsample_bytree': 0.7145526429288481, 'gamma': 2.5491566317456176, 'lambda': 7.955863255887215, 'alpha': 5.475375905704087, 'min_child_weight': 8}. Best is trial 0 with value: 0.7372457337175858.


Best trial: 0. Best value: 0.737246:   1%|▏         | 2/150 [00:09<12:42,  5.15s/it]C:\Users\Owner\AppData\Local\Temp\ipykernel_848\2214655158.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),


[I 2025-02-14 03:01:26,921] Trial 1 finished with value: 0.7237720938021754 and parameters: {'n_estimators': 400, 'max_depth': 12, 'learning_rate': 0.18985975946166567, 'subsample': 0.8616344202571158, 'colsample_bytree': 0.7673793695272236, 'gamma': 1.79132020385941, 'lambda': 4.180343711914006, 'alpha': 2.1609226333428144, 'min_child_weight': 2}. Best is trial 0 with value: 0.7372457337175858.


Best trial: 0. Best value: 0.737246:   2%|▏         | 3/150 [00:11<08:54,  3.64s/it]C:\Users\Owner\AppData\Local\Temp\ipykernel_848\2214655158.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),


...

[I 2025-02-14 03:37:48,729] Trial 147 finished with value: 0.737451736884483 and parameters: {'n_estimators': 800, 'max_depth': 6, 'learning_rate': 0.01593775468885407, 'subsample': 0.6311602039687986, 'colsample_bytree': 0.8146953353863756, 'gamma': 4.183342746669195, 'lambda': 8.689122455093235, 'alpha': 7.135985164187159, 'min_child_weight': 6}. Best is trial 128 with value: 0.7378580848542172.


Best trial: 128. Best value: 0.737858:  99%|█████████▉| 149/150 [36:46<00:14, 14.65s/it]C:\Users\Owner\AppData\Local\Temp\ipykernel_848\2214655158.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),


[I 2025-02-14 03:38:03,751] Trial 148 finished with value: 0.73739696491308 and parameters: {'n_estimators': 850, 'max_depth': 7, 'learning_rate': 0.012493497518939764, 'subsample': 0.9556514317134999, 'colsample_bytree': 0.7899824615790614, 'gamma': 1.8333206343737012, 'lambda': 8.313301207261484, 'alpha': 7.777231418740403, 'min_child_weight': 6}. Best is trial 128 with value: 0.7378580848542172.


Best trial: 128. Best value: 0.737858: 100%|██████████| 150/150 [37:02<00:00, 14.82s/it]

[I 2025-02-14 03:38:19,526] Trial 149 finished with value: 0.7377482808252039 and parameters: {'n_estimators': 850, 'max_depth': 7, 'learning_rate': 0.0110709508140582, 'subsample': 0.6220326480414409, 'colsample_bytree': 0.7810378502358201, 'gamma': 2.0346127606949804, 'lambda': 9.00840738699664, 'alpha': 7.8042262686899, 'min_child_weight': 6}. Best is trial 128 with value: 0.7378580848542172.


In [ ]:
import joblib

# Optuna 최적화 실행 후 최적의 study 저장
joblib.dump(study, "xgb_optuna_study.pkl")


['xgb_optuna_study.pkl']

In [19]:
import joblib

# 저장된 Optuna study 불러오기
study = joblib.load("xgb_optuna_study.pkl")

# 최적의 하이퍼파라미터 가져오기
best_params = study.best_params
print("Best Params:", best_params)

# 최적의 하이퍼파라미터로 모델 학습
model = xgb.XGBClassifier(**best_params, random_state=42, n_jobs=-1)
model.fit(X_train_encoded, y)


Best Params: {'n_estimators': 850, 'max_depth': 7, 'learning_rate': 0.012310242311885787, 'subsample': 0.6489034679243402, 'colsample_bytree': 0.8071605661865788, 'gamma': 4.3795131167061125, 'lambda': 8.995623491394046, 'alpha': 6.451638862845998, 'min_child_weight': 8}


XGBClassifier(alpha=6.451638862845998, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8071605661865788, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=4.3795131167061125,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=8.995623491394046,
              learning_rate=0.012310242311885787, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=8, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=850, n_jobs=-1, ...)

In [ ]:
# 최적 하이퍼파라미터 가져오기
best_params = study.best_params

# 최적의 파라미터로 모델 재학습
model = xgb.XGBClassifier(
    **best_params,
    objective="binary:logistic",
    eval_metric="auc",
    use_label_encoder=False,
    verbosity=0,
    random_state=42,
    n_jobs=-1
)

In [18]:
model.fit(X_train_encoded, y)

XGBClassifier(alpha=6.451638862845998, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8071605661865788, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=4.3795131167061125,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=8.995623491394046,
              learning_rate=0.012310242311885787, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=8, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=850, n_jobs=-1, ...)

In [20]:
import sklearn
import xgboost

print("Scikit-learn version:", sklearn.__version__)
print("XGBoost version:", xgboost.__version__)


Scikit-learn version: 1.6.1
XGBoost version: 2.1.4


In [21]:
from sklearn.metrics import accuracy_score, roc_auc_score

# 예측
y_train_pred = model.predict(X_train_encoded)
y_train_proba = model.predict_proba(X_train_encoded)[:, 1]  # ROC-AUC Score 계산용

# 평가
accuracy = accuracy_score(y, y_train_pred)
roc_auc = roc_auc_score(y, y_train_proba)

# 출력
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")

Accuracy: 0.7484
ROC-AUC Score: 0.7450


RandomForest (Hyperparameter tune)

In [22]:
model.save_model("xgboost_model2.json")

### Predict

In [23]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [24]:
sample_submission = pd.read_csv('./Data/sample_submission.csv')
sample_submission['probability'] = pred_proba

In [25]:
sample_submission.to_csv('./XGBoost2_optimization_submit.csv', index=False)